<a href="https://colab.research.google.com/github/PoeQulta/Train-Custom-Objects-Detector/blob/master/TrainObjects.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Future Implementations:


*   Train for multiple objects [✔]
*   TensorBoard functionality [✔]
*   use h5py to get over slow drive copy speeds



**First: create workspace and prepare environment for tf and object detection api**

In [0]:
%cd /content/
%mkdir TensorFlow
%cd TensorFlow
%mkdir workspace
!git clone https://github.com/tensorflow/models.git
!apt-get install protobuf-compiler python-pil python-lxml
!sudo pip install jupyter
!sudo pip install matplotlib
!sudo pip install pandas
%cd /content/TensorFlow/models/research/
!protoc object_detection/protos/*.proto --python_out=.
!export PYTHONPATH=$PYTHONPATH:`pwd`:`pwd`/slim
!python setup.py install
import os
os.environ['PYTHONPATH'] += ':/content/Tensorflow/models/research/:/content/TensorFlow/models/research/slim/'
%cd /content/TensorFlow/workspace/
%mkdir training_objects
%cd training_objects
%mkdir annotations
%mkdir pre-trained-model
%mkdir images
%mkdir training
%cd images
%mkdir train
%mkdir test

See https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/training.html
![alt text](https://i.ibb.co/c8FNwVt/Screenshot-from-2020-03-07-18-20-05.png)

objectNames is a list of labels in dataset

Edit accordingly

In [0]:
global objectName
objectNames = ['square','circle','coral','star']

**Get  a pretrained model**, Mobilenet ssd in this case, trained on coco

In [0]:
%cd /content/TensorFlow/workspace/training_objects/pre-trained-model
!wget http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v3_small_coco_2019_08_14.tar.gz
!tar -xvf ssd_mobilenet_v3_small_coco_2019_08_14.tar.gz

/content/TensorFlow/workspace/training_objects/pre-trained-model
--2020-03-25 19:42:18--  http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v3_small_coco_2019_08_14.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 172.217.3.16, 2607:f8b0:4026:802::2010
Connecting to download.tensorflow.org (download.tensorflow.org)|172.217.3.16|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 35910144 (34M) [application/x-tar]
Saving to: ‘ssd_mobilenet_v3_small_coco_2019_08_14.tar.gz’

ssd_mobilenet_v3_sm 100%[===================>]  34.25M   136MB/s    in 0.3s    

2020-03-25 19:42:18 (136 MB/s) - ‘ssd_mobilenet_v3_small_coco_2019_08_14.tar.gz’ saved [35910144/35910144]

ssd_mobilenet_v3_small_coco_2019_08_14/
ssd_mobilenet_v3_small_coco_2019_08_14/model.ckpt.index
ssd_mobilenet_v3_small_coco_2019_08_14/checkpoint
ssd_mobilenet_v3_small_coco_2019_08_14/model.ckpt.meta
ssd_mobilenet_v3_small_coco_2019_08_14/frozen_inference_graph.pb
ssd_mob

Mount Drive to get datasets from and save frozen graph into

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Grab images & xmls into the training folder (includes files in subdirectories)

*   change first path to dataset path



In [0]:
!find "/content/Data/" -type f -print0 | xargs -0 mv -t /content/TensorFlow/workspace/training_objects/images/train/

Below code checks for corrupt files 

In [0]:
import tensorflow as tf
from os import listdir
from PIL import Image
direc = "/content/TensorFlow/workspace/training_objects/images/test/"
count = 0
for filename in listdir(direc):
  fn = direc+filename
  if filename.endswith('.jpg'):
    try:
      with tf.Graph().as_default():
        image_contents = tf.read_file(fn)
        image = tf.image.decode_jpeg(image_contents, channels=3)
        init_op = tf.initialize_all_tables()
        with tf.Session() as sess:
          sess.run(init_op)
          tmp = sess.run(image)
    except:
      try:
        os.remove(fn[:-3]+'xml')
        os.remove(fn[:-3]+'jpg')
      except:
        print("doesn't exist")
      print(fn)

Picks 50 random test samples from dataset 

In [0]:
import os, random
sourceDir = "/content/TensorFlow/workspace/training_objects/images/train/"
destDir = "/content/TensorFlow/workspace/training_objects/images/test/"
for i in range(50):
  fileName = random.choice(os.listdir(sourceDir))
  fileName = fileName[:-3]
  path = os.path.join(sourceDir, fileName)
  destPath = os.path.join(destDir, fileName)
  sourcePathItems = [path+'jpg',path+'xml']
  destPathItems = [destPath+'jpg',destPath+'xml']
  #print(pathJpg + ":"+pathxml)
  try:
    os.rename(sourcePathItems[0], destPathItems[0])
    os.rename(sourcePathItems[1], destPathItems[1])
  except:
    print(destPathItems[1])

**After images are placed in Tensorflow/workspace/training_/images/train and /test appropriatly then:**

Convert xml annotation files to CSV

In [0]:
import os
import glob
import pandas as pd
import xml.etree.ElementTree as ET

def xml_to_csv(path):
    """Iterates through all .xml files (generated by labelImg) in a given directory and combines them in a single Pandas datagrame.

    Parameters:
    ----------
    path : {str}
        The path containing the .xml files
    Returns
    -------
    Pandas DataFrame
        The produced dataframe
    """

    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            value = (path+root.find('filename').text,
                    int(root.find('size')[0].text),
                    int(root.find('size')[1].text),
                    member[0].text,
                    int(member[4][0].text),
                    int(member[4][1].text),
                    int(member[4][2].text),
                    int(member[4][3].text)
                    )
            xml_list.append(value)
    column_name = ['filename', 'width', 'height',
                'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df


def main():
    inputDir = '/content/TensorFlow/workspace/training_'+objectName+'/images/train/'
    outpuDir = '/content/TensorFlow/workspace/training_'+objectName+'/annotations/train.csv'
    xml_df = xml_to_csv(inputDir)
    xml_df.to_csv(
        outpuDir, index=None)
    print('Successfully converted training xml to csv.')
    inputDir = "/content/TensorFlow/workspace/training_"+objectName+"/images/test/"
    outpuDir = "/content/TensorFlow/workspace/training_"+objectName+"/annotations/test.csv"
    xml_df = xml_to_csv(inputDir)
    xml_df.to_csv(
        outpuDir, index=None)
    print('Successfully converted test xml to csv.')

if __name__ == '__main__':
    main()

**Convert csv to .record
get script that converts from csv to .record**
*for some bizarro reason, this py script breaks if run as a cell, but from the virtual shell, no exceptions are thrown---probably an Ipython compatibility issue*

In [0]:
%cd /content/
!curl -L -o generateRecord.py "https://drive.google.com/uc?export=download&id=1I_JILFK7RnOFUf2LcpNNtdU79Q2eX91B"

convert csv to .record

In [0]:
!python generateRecord.py --csv_input=/content/TensorFlow/workspace/training_objects/annotations/train.csv --output_path=/content/TensorFlow/workspace/training_objects/annotations/train.record
!python generateRecord.py --csv_input=/content/TensorFlow/workspace/training_objects/annotations/test.csv --output_path=/content/TensorFlow/workspace/training_objects/annotations/test.record

Pull configuration file for the model used

In [0]:
%cd /content/TensorFlow/workspace/training_objects/training/
!curl -L -o ssd_mobilenet_v3_pets.config "https://drive.google.com/uc?export=download&id=1IwgZ4sGvIJvUjK-t70r66CeVkblyF8W-"

/content/TensorFlow/workspace/training_objects/training
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   388    0   388    0     0    994      0 --:--:-- --:--:-- --:--:--   992
100  4628  100  4628    0     0   6010      0 --:--:-- --:--:-- --:--:--  6010


Edit labelmap and configuration file for one object **for multiple objects the files have to be edited manually**   will fix in future versions 

In [0]:
with open('/content/TensorFlow/workspace/training_objects/training/labelmap.pbtxt', 'w') as file:
  for i,obj in enumerate(objectNames):
    file.writelines( 'item {\n id: '+str(i+1)+' \n display_name:"'+obj+'"\n}')
with open('/content/TensorFlow/workspace/training_objects/training/ssd_mobilenet_v3_pets.config', 'r') as file:
    # read a list of lines into data
    data = file.readlines()
data[181] = 'input_path:"/content/TensorFlow/workspace/training_objects/annotations/train.record"'
data[183] = 'label_map_path: "/content/TensorFlow/workspace/training_objects/training/labelmap.pbtxt"'
data[192] = 'input_path:"/content/TensorFlow/workspace/training_objects/annotations/test.record"'
data[194] = 'label_map_path: "/content/TensorFlow/workspace/training_objects/training/labelmap.pbtxt"'
with open('/content/TensorFlow/workspace/training_objects/training/ssd_mobilenet_v3_pets.config', 'w') as file:
    file.writelines( data )

Optionally run TensorBoard using ngork


---



In [0]:
%cd /content/
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
LOG_DIR = '/content/TensorFlow/workspace/training_objects/training/'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)
get_ipython().system_raw('./ngrok http 6006 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"



---
Start training:


In [0]:
!python /content/TensorFlow/models/research/object_detection/legacy/train.py --logtostderr --pipeline_config_path=/content/TensorFlow/workspace/training_objects/training/ssd_mobilenet_v3_pets.config --train_dir=/content/TensorFlow/workspace/training_Mapping/training/ --log

In [0]:
!python /content/TensorFlow/models/research/object_detection/export_inference_graph.py \
    --input_type image_tensor \
    --pipeline_config_path /content/TensorFlow/workspace/training_objects/training/ssd_mobilenet_v3_pets.config\
    --trained_checkpoint_prefix /content/TensorFlow/workspace/training_objects/training/model.ckpt-391\
    --output_directory /content/Mapping/

In [0]:
!rm /content/TensorFlow/workspace/training_Mapping/images/train/*

Move images from a directory to another

In [0]:
import os, random
sourceDir = "/content/combined/"
destDir = "/content/TensorFlow/workspace/training_Mapping/images/train/"
for i in os.listdir(sourceDir):
  fileName = i
  path = os.path.join(sourceDir, fileName)
  destPath = os.path.join(destDir, fileName)
  sourcePathItems = [path+'jpg',path+'xml']
  destPathItems = [destPath+'jpg',destPath+'xml']
  #print(pathJpg + ":"+pathxml)
  os.rename(path, destPath)

removes duplicates

In [0]:
from os import listdir
direc = "/content/combined/"
count = 0
for filename in listdir(direc):
  #print(len(listdir(direc)))
  if filename[-7:] == "(1).jpg":
    print('Bad file:', filename) # print out the names of corrupt files
    os.remove(direc+filename)
    count +=1
print(count)

In [0]:
!cp -a -v "/content/drive/My Drive/DataSets/Star/". /content/combined/
!cp -a -v "/content/drive/My Drive/DataSets/Circle/". /content/combined/
!cp -a -v "/content/drive/My Drive/DataSets/Designated_Area1_labeld/". /content/combined/
!cp -a -v "/content/drive/My Drive/DataSets/Designated_Area2_labled/". /content/combined/
!cp -a -v "/content/drive/My Drive/DataSets/Coral_fragment_2_labeled/". /content/combined/